In [1]:
import os
import csv
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.utils import to_categorical

# x  - timeStamp
# y  - standID

filepath = 'train.csv'
data = pd.read_csv(filepath)    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
b = data.loc[data['CALL_TYPE']=='B']
no_nan = b.loc[b['ORIGIN_STAND'].isnull() == False]
not_missing = no_nan.loc[no_nan['MISSING_DATA'] == False]
data = not_missing
tempData = data.drop(columns = ["TRIP_ID", "CALL_TYPE", "ORIGIN_CALL", "DAY_TYPE", "MISSING_DATA"])
data = tempData

In [3]:
def findsecondlast(text, pattern):
    return text.rfind(pattern, 0, text.rfind(pattern))

In [4]:
npdata = data.as_matrix()
#print(npdata.shape)
tripTime = np.empty(shape=(806576,1))
tempdata = npdata[:, 3].tolist()
tempdata = [a.replace("[", "").replace("]","") for a in tempdata]
#print(len(tempdata))
for i, string in enumerate(tempdata):
    tripTime[i] = (string.count(',')+1)/2.0 * 15
    secondlastcomma = findsecondlast(string, ',') + 1
    lastcor = np.fromstring(string[secondlastcomma:], dtype = 'float32', sep = ',')
    tempdata[i] = lastcor

tempdata = np.array(tempdata)
npdata[:,3] = tempdata
print(npdata.shape)
print(tripTime.shape)
npdata = np.append(npdata, tripTime, axis = 1)
print(npdata.shape)
print(npdata)

(806576, 4)
(806576, 1)
(806576, 5)
[[7.0 20000596 1372637303 array([-8.66574, 41.17067], dtype=float32)
  285.0]
 [13.0 20000497 1372637610 array([-8.628147, 41.15752 ], dtype=float32)
  960.0]
 [28.0 20000403 1372638481 array([-8.641566, 41.142673], dtype=float32)
  975.0]
 ...
 [9.0 20000121 1404155105 array([-8.67015 , 41.236866], dtype=float32)
  1065.0]
 [12.0 20000248 1404141826 array([-8.587026, 41.173523], dtype=float32)
  930.0]
 [34.0 20000079 1404157147 array([-8.620893, 41.14988 ], dtype=float32)
  405.0]]


In [19]:
#sorting based on driverID, timestamp
indices = np.lexsort((npdata[:, 2], npdata[:, 1]))
print(indices.shape)
sortdata = npdata[indices]
print(sortdata)
npdata = sortdata

(806576,)
[[28.0 20000001 1372662403 array([-8.607537, 41.162285], dtype=float32)
  300.0]
 [63.0 20000001 1372669154 array([-8.593308, 41.147964], dtype=float32)
  540.0]
 [10.0 20000001 1372676157 array([-8.565732, 41.174713], dtype=float32)
  780.0]
 ...
 [61.0 20000904 1404160289 array([-8.623017, 41.15016 ], dtype=float32)
  330.0]
 [61.0 20000904 1404164541 array([-8.579934, 41.159485], dtype=float32)
  645.0]
 [53.0 20000904 1404165869 array([-8.606925, 41.152473], dtype=float32)
  390.0]]


In [6]:
sortdata2 = npdata.copy()
for i in range(0,806575):
    sortdata2[i, 0] = sortdata2[i+1, 0]
for i in range(806575,0, -1):
    sortdata2[i, 3] = sortdata2[i-1, 3]
print(sortdata2)
    
    

[[63.0 20000001 1372662403 array([-8.607537, 41.162285], dtype=float32)
  300.0]
 [10.0 20000001 1372669154 array([-8.607537, 41.162285], dtype=float32)
  540.0]
 [25.0 20000001 1372676157 array([-8.593308, 41.147964], dtype=float32)
  780.0]
 ...
 [61.0 20000904 1404160289 array([-8.574624, 41.15666 ], dtype=float32)
  330.0]
 [53.0 20000904 1404164541 array([-8.623017, 41.15016 ], dtype=float32)
  645.0]
 [53.0 20000904 1404165869 array([-8.579934, 41.159485], dtype=float32)
  390.0]]


In [7]:
cleandata = np.delete(sortdata2, 0, 0)
cleandata = np.delete(cleandata, 806574, 0)
firstDriver = cleandata[0, 1]
not_to_delete = np.ones(806574)
for i in range(0,806573):
    if cleandata[i, 1] != firstDriver:
        firstDriver = cleandata[i, 1]
        not_to_delete[i] = 0
        not_to_delete[i-1] = 0

print(806574 - np.sum(not_to_delete))
not_to_delete = not_to_delete.astype('bool')
cleandata = cleandata[not_to_delete]
print(cleandata.shape)
print(cleandata)

876.0
(805698, 5)
[[10.0 20000001 1372669154 array([-8.607537, 41.162285], dtype=float32)
  540.0]
 [25.0 20000001 1372676157 array([-8.593308, 41.147964], dtype=float32)
  780.0]
 [30.0 20000001 1372683156 array([-8.565732, 41.174713], dtype=float32)
  825.0]
 ...
 [61.0 20000904 1404157116 array([-8.59797, 41.15439], dtype=float32)
  465.0]
 [61.0 20000904 1404160289 array([-8.574624, 41.15666 ], dtype=float32)
  330.0]
 [53.0 20000904 1404164541 array([-8.623017, 41.15016 ], dtype=float32)
  645.0]]


In [8]:
print(np.mean(cleandata[:,4]))

678.8493796683124


In [9]:
not_to_delete = np.ones(805698)
firstDriver = cleandata[0, 1]

for i in range(1, 805698):
    if cleandata[i,1] != firstDriver:
        firstDriver = cleandata[i,1]
        continue
    
    if cleandata[i, 2] - cleandata[i-1, 2] - cleandata[i-1,4] > 8839:   #check if the timestamps differ by more than 30 minutes
        not_to_delete[i] = 0

print(np.sum(not_to_delete))
cleandata2 = cleandata.copy()
not_to_delete = not_to_delete.astype(np.bool)
cleandata2 = cleandata2[not_to_delete]
print(cleandata2.shape)
print(cleandata2)       
        

505328.0
(505328, 5)
[[10.0 20000001 1372669154 array([-8.607537, 41.162285], dtype=float32)
  540.0]
 [25.0 20000001 1372676157 array([-8.593308, 41.147964], dtype=float32)
  780.0]
 [30.0 20000001 1372683156 array([-8.565732, 41.174713], dtype=float32)
  825.0]
 ...
 [61.0 20000904 1404071729 array([-8.612955, 41.137722], dtype=float32)
  345.0]
 [61.0 20000904 1404160289 array([-8.574624, 41.15666 ], dtype=float32)
  330.0]
 [53.0 20000904 1404164541 array([-8.623017, 41.15016 ], dtype=float32)
  645.0]]


In [52]:
sorted_indices = np.lexsort((cleandata2[:,2], cleandata2[:,1]))
print(sorted_indices.shape)
sorted_by_timestamp = cleandata2[sorted_indices]
# to_order = cleandata2.copy()
# dtype = [("stand", np.int), ("driver", np.int), ("timestamp", np.int), ("pos", np.ndarray), ("drive_time", np.int)]
# print(to_order.tolist())
# to_order = np.array(to_order.tolist(), dtype=dtype)
# to_order.astype(dtype)
# np.append([['stand', 'driver', 'timestamp', 'pos', 'drive_time']],to_order, axis=0)
# np.sort(to_order, order='timestamp')
# print(to_order)
# print(sorted_by_timestamp)
first_timestamp = sorted_by_timestamp[0][2]
start_index = 0
end_index = 0
for i in range(sorted_by_timestamp.shape[0]):
#     print(sorted_by_timestamp[i][2])
    if sorted_by_timestamp[i][2] - first_timestamp < 1800:
        end_index = i
    else:
#         print(start_index, end_index)
#         print(first_timestamp, sorted_by_timestamp[i][2])
        if(start_index == end_index):
            start_index = end_index + 1
            first_timestamp = sorted_by_timestamp[i][2]
            continue
        print(start_index - end_index)
        stands = sorted_by_timestamp[start_index:end_index, 0]
        stands = stands.astype(np.int)
        my = np.bincount(stands)
        ii = np.nonzero(my)[0]
        counts = np.vstack((ii, my[ii])).T
        if counts.size != 0:
            print(counts)

        start_index = end_index + 1
        first_timestamp = sorted_by_timestamp[i][2]

(504305,)
1
1
1
-4
[[19  1]
 [25  1]
 [30  1]
 [61  1]]
1
1
1
1
1
1
1
-8
[[ 7  1]
 [16  1]
 [19  3]
 [25  1]
 [30  1]
 [37  1]]
1
1
1
1
1
1
1
1
1
1
1
-12
[[ 7  1]
 [10  1]
 [19  4]
 [25  1]
 [28  1]
 [42  2]
 [58  2]]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-22
[[16  1]
 [17  1]
 [19  4]
 [25  2]
 [26  1]
 [28  5]
 [29  1]
 [30  1]
 [37  1]
 [40  1]
 [52  1]
 [53  2]
 [63  1]]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-17
[[10  1]
 [12  1]
 [15  1]
 [19  5]
 [23  1]
 [25  1]
 [27  2]
 [28  4]
 [34  1]]
1
-2
[[19  2]]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-21
[[ 7  1]
 [10  1]
 [17  1]
 [19  8]
 [22  1]
 [25  1]
 [28  3]
 [30  2]
 [36  1]
 [37  1]
 [62  1]]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-29
[[10  1]
 [12  1]
 [19  6]
 [28  4]
 [29  1]
 [30  2]
 [32  1]
 [36  2]
 [37  1]
 [47  1]
 [51  1]
 [52  1]
 [56  1]
 [58  1]
 [62  4]
 [63  1]]
1
1
1
1
1
1
1
1
-9
[[10  1]
 [17  1]
 [28  3]
 [30  1]
 [39  1]
 [52  1]
 [62  1]]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

In [10]:
'''total_time = 0
firstDriver = cleandata[0, 1]
things = np.zeros(shape=(805698,))

num = 0
for i in range(1, 805698):
    if cleandata[i,1] != firstDriver:
        firstDriver = cleandata[i,1]
        continue
    if cleandata[i, 2] - cleandata[i-1, 2] - cleandata[i-1,4] < 8839:
        num += 1
        total_time += cleandata[i, 2] - cleandata[i-1, 2]  - cleandata[i-1, 4]
        things[i] = cleandata[i, 2] - cleandata[i-1, 2]  - cleandata[i-1, 4]

things = things[things != 0]
# bad_stuff = things <= 3 * 16096
# things = things[bad_stuff]

print(things.shape)
print(np.mean(things))
print(things)
print(np.std(things))
print(num)
'''


'total_time = 0\nfirstDriver = cleandata[0, 1]\nthings = np.zeros(shape=(805698,))\n\nnum = 0\nfor i in range(1, 805698):\n    if cleandata[i,1] != firstDriver:\n        firstDriver = cleandata[i,1]\n        continue\n    if cleandata[i, 2] - cleandata[i-1, 2] - cleandata[i-1,4] < 8839:\n        num += 1\n        total_time += cleandata[i, 2] - cleandata[i-1, 2]  - cleandata[i-1, 4]\n        things[i] = cleandata[i, 2] - cleandata[i-1, 2]  - cleandata[i-1, 4]\n\nthings = things[things != 0]\n# bad_stuff = things <= 3 * 16096\n# things = things[bad_stuff]\n\nprint(things.shape)\nprint(np.mean(things))\nprint(things)\nprint(np.std(things))\nprint(num)\n'

In [11]:
ctr = 0
temparray = np.ones(shape = (cleandata2.shape[0], ))
for i in range(cleandata2.shape[0]):
    if cleandata2[i, 3].shape[0] == 0:
        temparray[i] = 0

print(cleandata2.shape)    
temparray = temparray.astype('bool')
cleandata2 = cleandata2[temparray]
print(cleandata2.shape)        
print(ctr)

(505328, 5)
(504305, 5)
0


In [12]:
import datetime

x_train = np.empty(shape = (504305,), dtype='object')
location = np.empty(shape = (504305,2))
for i in range(504305):
    loc = cleandata2[i, 3]
    lat = loc[0]
    lon = loc[1]
    location[i, 0] = lat
    location[i, 1] = lon
x_train = x_train[:, np.newaxis]
x_train = np.append(x_train, location, axis = 1)
# print(x_train.shape)
y_train = cleandata2[:, 0].astype('int32')
print(y_train.shape)

timee = np.empty(shape = (504305,4), dtype='int32')

for i in range(cleandata2.shape[0]):
    ts = datetime.datetime.fromtimestamp(cleandata2[i][2])
    timee[i, 0] = ts.month
    timee[i, 1] = ts.day
    timee[i, 2] = ts.hour
    timee[i, 3] = ts.minute



x_train = np.delete(x_train, 0, 1)
x_train = np.append(x_train, timee, axis = 1)
# x_train = x_train.astype('float32')
print(x_train.shape)
print(x_train)

    
    

(504305,)
(504305, 6)
[[-8.607537269592285 41.16228485107422 7 1 16 59]
 [-8.593308448791504 41.14796447753906 7 1 18 55]
 [-8.5657320022583 41.174713134765625 7 1 20 52]
 ...
 [-8.612955093383789 41.13772201538086 6 30 3 55]
 [-8.574624061584473 41.15665817260742 7 1 4 31]
 [-8.623017311096191 41.15016174316406 7 1 5 42]]


In [13]:
y_train = to_categorical(y_train)
y_train = np.delete(y_train, 0 ,1)
print(y_train.shape)
print(y_train)

(504305, 63)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
print(x_train[0])

[-8.607537269592285 41.16228485107422 7 1 16 59]


In [56]:

def stand_to_img_coords(stand_id):
    pass

over_x = cleandata2[:10000,:]

sorted_indices = np.lexsort((over_x[:,2], over_x[:,1]))
print(sorted_indices.shape)
sorted_by_timestamp = cleandata2[sorted_indices]
# to_order = cleandata2.copy()
# dtype = [("stand", np.int), ("driver", np.int), ("timestamp", np.int), ("pos", np.ndarray), ("drive_time", np.int)]
# print(to_order.tolist())
# to_order = np.array(to_order.tolist(), dtype=dtype)
# to_order.astype(dtype)
# np.append([['stand', 'driver', 'timestamp', 'pos', 'drive_time']],to_order, axis=0)
# np.sort(to_order, order='timestamp')
# print(to_order)
# print(sorted_by_timestamp)
first_timestamp = sorted_by_timestamp[0][2]
start_index = 0
end_index = 0
for i in range(sorted_by_timestamp.shape[0]):
#     print(sorted_by_timestamp[i][2])
    if sorted_by_timestamp[i][2] - first_timestamp < 1800:
        end_index = i
    else:
#         print(start_index, end_index)
#         print(first_timestamp, sorted_by_timestamp[i][2])
        if(start_index == end_index):
            start_index = end_index + 1
            first_timestamp = sorted_by_timestamp[i][2]
            continue
#         print(start_index - end_index)
        stands = sorted_by_timestamp[start_index:end_index, 0]
        stands = stands.astype(np.int)
        my = np.bincount(stands)
        ii = np.nonzero(my)[0]
        counts = np.vstack((ii, my[ii])).T
        if counts.size != 0:
            #do the computation relating to putting the largest stands on the map
            
            pass

        start_index = end_index + 1
        first_timestamp = sorted_by_timestamp[i][2]

(10000,)
6 [[19  1]
 [25  1]
 [30  1]
 [61  1]]
15 [[ 7  1]
 [16  1]
 [19  3]
 [25  1]
 [30  1]
 [37  1]]
28 [[ 7  1]
 [10  1]
 [19  4]
 [25  1]
 [28  1]
 [42  2]
 [58  2]]
51 [[16  1]
 [17  1]
 [19  4]
 [25  2]
 [26  1]
 [28  5]
 [29  1]
 [30  1]
 [37  1]
 [40  1]
 [52  1]
 [53  2]
 [63  1]]
69 [[10  1]
 [12  1]
 [15  1]
 [19  5]
 [23  1]
 [25  1]
 [27  2]
 [28  4]
 [34  1]]
72 [[19  2]]
94 [[ 7  1]
 [10  1]
 [17  1]
 [19  8]
 [22  1]
 [25  1]
 [28  3]
 [30  2]
 [36  1]
 [37  1]
 [62  1]]
124 [[10  1]
 [12  1]
 [19  6]
 [28  4]
 [29  1]
 [30  2]
 [32  1]
 [36  2]
 [37  1]
 [47  1]
 [51  1]
 [52  1]
 [56  1]
 [58  1]
 [62  4]
 [63  1]]
134 [[10  1]
 [17  1]
 [28  3]
 [30  1]
 [39  1]
 [52  1]
 [62  1]]
163 [[ 7  1]
 [12  1]
 [19 10]
 [28  7]
 [30  3]
 [31  1]
 [34  2]
 [36  1]
 [63  2]]
174 [[10  1]
 [19  2]
 [28  1]
 [35  1]
 [42  3]
 [53  1]
 [58  1]]
182 [[19  2]
 [28  3]
 [31  1]
 [63  1]]
202 [[ 6  1]
 [10  1]
 [19  4]
 [25  2]
 [28  3]
 [30  2]
 [31  1]
 [35  2]
 [55  1]
 [56  1]

In [ ]:
import matplotlib.pyplot as plt
import keras.optimizers
import keras
from keras.callbacks import ModelCheckpoint

print(x_train[0])
over_x = x_train[:10000,]
# meow = np.zeros(63)
# meow[5] = 1
over_y = y_train[:10000,]
print(over_x.shape)
print(over_y.shape)
print(over_x)
print(over_y)

# print(x_train.shape)
model = Sequential()
# model.add(Dense(10000, input_shape = (3,), activation  = 'relu'))
model.add(Dense(128, input_dim=6, activation  = 'relu', kernel_initializer='glorot_uniform'))
model.add(Dense(256, kernel_initializer='glorot_uniform', activation  = 'relu'))
model.add(Dense(1024, kernel_initializer='glorot_uniform', activation  = 'relu'))
# model.add(Dense(2048, kernel_initializer='glorot_uniform', activation  = 'relu'))
# model.add(Dense(64))
#model.add(Flatten())
model.add(Dense(63, activation = 'softmax'))
model.summary()

# model.summary()
adam = keras.optimizers.Adam(lr=0.0019)#lr = 0.0019
# sgd = keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
# model.compile(loss = keras.losses.categorical_crossentropy, optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True), metrics = ['accuracy'])
model.compile(loss = keras.losses.categorical_crossentropy, optimizer = adam, metrics = ['accuracy'])
# history = model.fit(x_train, y_train, validation_split = 0.1, epochs = 10, batch_size = 128)
# history = model.fit(over_x, over_y, validation_split = 0.1, epochs = 1000, batch_size = 512, verbose=1)
history = model.fit(over_x, over_y, epochs = 1000, batch_size = 512, verbose=1)
model.save('final_weights.h5')
plt.subplot(211)
plt.title("LOSS")
plt.plot(history.history['acc'])
plt.plot(history.history['loss'])

In [ ]:
print(model.predict([1,1,1,1,1,1]))